In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


**La Y representa el porcentaje que cobra un jugador respecto al presupuesto del equipo.**

Los datos no son los de la temporada 2019, son la media de todas las temporadas que el jugador ha disputado en la NBA

Los salarios corresponden a la temporada 2020

In [ ]:
df_nor=pd.read_csv("/kaggle/input/nba-salarios/df_kaggle.csv")
df_nor=df_nor[df_nor.MP_total>df_nor.MP_total.quantile(0.1)]#Elimina el 25% que menos juega
df_nor=df_nor.dropna()

Se elimina el 10% de los jugadores que menos han jugado


In [ ]:
sns.kdeplot(data=df_nor, x="Y")


Observamos que la funcion de probabilidad de la variable Y sigue una especie de distribucion lognormal

In [ ]:
df_salario=df_nor.groupby("Age_total").mean().reset_index()

plt.figure(figsize=(15,8))
sns.barplot(x="Age_total", y="Y", data=df_salario, orient="v")

Distribucion de la Y según la edad

Evidentemente a los mas novatos se les paga menos

In [ ]:
df_nor.Y.describe().to_frame()

In [ ]:
df_nor.loc[df_nor.Y.idxmax()].Player_total #El jugador que mas Y tiene es Blake Griffin en su etapa en detroit

In [ ]:
df_nor.drop(["Salario","Salario_a"],axis=1,inplace=True)
mejor_corr=df_nor.corr().loc["Y"].sort_values().dropna().tail(10).keys()

In [ ]:
df_corr=df_nor[list(mejor_corr)]

correlation_mat = df_corr.corr()

plt.figure(figsize=(15, 8))

sns.heatmap(correlation_mat, annot = True)

Las variables que mas se correlacionan con la Y y sus R^2 

Observamos el Value Over Replacement es la mas correlacionada

In [ ]:
plt.figure(figsize=(15, 8))

sns.pairplot(df_corr,y_vars=['Y'])

In [ ]:
sns.boxplot(x="Pos_total", y="VORP_Advanced", data=df_nor).set(
    xlabel='Posicion', 
    ylabel='VORP')

La posicion no parece influir en exceso en la decisividad de un jugador

In [ ]:
import statsmodels.api as sm

X=df_corr.drop("Y",axis=1)
y=df_corr["Y"]


In [ ]:
X = sm.add_constant(X)

modelo = sm.OLS(endog=y, exog=X,)
modelo = modelo.fit()
print(modelo.summary())

Se pueden eliminar varias variables que no aportan info al modelo (MP,TOV,FG), ademas existe colinealidad (variables que se correlacionan entre si) seguramente entre FG y FGA. Aun asi el modelo obtiene un R^2 de 0.63 que no esta nada mal debido a la simpleza y los pocos datos que se tienen


Este notebook es una pequeñisima demostracion del potencial de python para la visualizacion de datos, como en unas simples lineas de codigo se puede extraer una gran cantidad de información. 

Este notebook forma parte de la visualizacion de datos del TFG que estoy realizando en el que con una base de datos mucho mas grande y con modelos de machine learning se trata de predecir la Y